In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts="cmdsmacods02", port=9200, timeout=30)
import csv
import numbers
import datetime
import pandas as pd
df4=pd.read_csv("/Users/vanisahu/Desktop/incaretest2.csv")
df5= pd.read_csv('/Users/vanisahu/Desktop/pd_incare.csv', error_bad_lines=False)
print 'length of dataframe is',len(df5)

#The red error of skipping lines will disappear when we will take the data directly from ES
#df5=pd.read_csv("/Users/vanisahu/Desktop/pd_incare.csv")
#print df4.head(3)

length of dataframe is 3968


Skipping line 125: expected 21 fields, saw 22
Skipping line 130: expected 21 fields, saw 27
Skipping line 270: expected 21 fields, saw 22
Skipping line 638: expected 21 fields, saw 22
Skipping line 674: expected 21 fields, saw 23
Skipping line 841: expected 21 fields, saw 52
Skipping line 1128: expected 21 fields, saw 22
Skipping line 1152: expected 21 fields, saw 23
Skipping line 1319: expected 21 fields, saw 23
Skipping line 1528: expected 21 fields, saw 24
Skipping line 1532: expected 21 fields, saw 23
Skipping line 1568: expected 21 fields, saw 26
Skipping line 1574: expected 21 fields, saw 23
Skipping line 1712: expected 21 fields, saw 22
Skipping line 1805: expected 21 fields, saw 31
Skipping line 2210: expected 21 fields, saw 22
Skipping line 2406: expected 21 fields, saw 23
Skipping line 2541: expected 21 fields, saw 22
Skipping line 2760: expected 21 fields, saw 30
Skipping line 2767: expected 21 fields, saw 22
Skipping line 2787: expected 21 fields, saw 23
Skipping line 2943:

In [175]:
from datetime import datetime
import numpy as np
def epoch_convert(date2):
    date1='1/1/1970'
    #date=raw_input('Enter date in mm/dd/yy format')
    #change date1 to date when you want input from user to be taken directly
    start_date = datetime.strptime(date1, "%m/%d/%Y")
    end_date = datetime.strptime(date2, "%m/%d/%Y")
    #print 'end date is',end_date
    days= abs((end_date-start_date).days)
    epoch_stamp=days*86400*1000
    return epoch_stamp
raw_start_date=raw_input('Enter start date in mm/dd/yyyy format')
#print type(raw_start_date)
user_start_date=epoch_convert(raw_start_date)
print 'user start date is',user_start_date
raw_end_date=raw_input('Enter end date in mm/dd/yyyy format')
user_end_date=epoch_convert(raw_end_date)
print 'user end date is',user_end_date

datetime_start = datetime.strptime(raw_start_date, '%m/%d/%Y')
datetime_end= datetime.strptime(raw_end_date, '%m/%d/%Y')
num_days= (datetime_end-datetime_start).days
print 'Total number of days in the given range are',num_days
days = np.busday_count( datetime_start,datetime_end )
print 'Number of working days in the given date range are',days
#print (datetime_end-datetime_start).days()


Enter start date in mm/dd/yyyy format1/1/2017
user start date is 1483228800000
Enter end date in mm/dd/yyyy format12/12/2017
user end date is 1513036800000
Total number of days in the given range are 345
Number of working days in the given date range are 246


In [176]:
def care_user_touched(user_start_date,user_end_date,df):
    df=df[(df.activity_timestamp>=user_start_date) &(df.activity_timestamp<=user_end_date)]
    #take this from a csv or a list predefined
    #unique=['user_293','user_161']
    unique=df['user_id'].unique().tolist()
    #df2=df.user_id.isin(unique)
    #df2=df[(df.user_id is in unique)]
    df2 =df[df.user_id.isin(unique)]
    df2=df2.groupby(by='user_id', as_index=False).agg({'empi': pd.Series.nunique})
    df2 = df2.sort_values(by=['empi'], ascending=[False])
    df2 = df2.drop(df2[df2.empi < 5].index)
    #pd.Series([np.nan]).sum(min_count=1)
    total=df2['empi'].sum()
    count=df2['user_id'].count()
    #sum_empi=df2.sum('empi')
    #df2=df2.sort(['empi'], ascending=[1])
    return total/count
print 'average number of unique patients touched upon are',care_user_touched(user_start_date,user_end_date,df4)



average number of unique patients touched upon are 6


In [177]:
#print df5.head(3)
def care_activities(user_start_date,user_end_date,df):
    df=df[(df.resource_type=='timeline')]
    unique=df['user'].unique().tolist()
    df2 =df[df.user.isin(unique)]
    df2=df2.groupby(by='user', as_index=False).agg({'empi': pd.Series.nunique})
    df2 = df2.drop(df2[df2.empi < 5].index)
    total=df2['empi'].sum()
    count=df2['user'].count()
    return ((total/count)*100)/days

print 'Number of average care activities per coordinator per day are',care_activities(user_start_date,user_end_date,df5)

Number of average care activities per coordinator per day are 18


In [ ]:
from ipywidgets import widgets
from IPython.display import display
import requests
res = requests.get('http://cmdsmacods02:9200')
print(res.content)

In [3]:
#es.get(index='pd_care_management', id=2)
es.search(index='pd_care_management', q='resource_type:"timeline"')

ConnectionError: ConnectionError((<urllib3.connection.HTTPConnection object at 0x11198c190>, u'Connection to cmdsmacods02 timed out. (connect timeout=30)')) caused by: ConnectTimeoutError((<urllib3.connection.HTTPConnection object at 0x11198c190>, u'Connection to cmdsmacods02 timed out. (connect timeout=30)'))

In [7]:
import requests
res = requests.get('http://localhost:9200')
print(res.content)

ConnectionError: HTTPConnectionPool(host='localhost', port=9200): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11b02cad0>: Failed to establish a new connection: [Errno 61] Connection refused',))